In [8]:
import telegram
from telegram.ext import Updater
from telegram.ext import CommandHandler
from telegram.ext import MessageHandler, Filters
import requests 
import json
import pandas as pd
from bs4 import BeautifulSoup
import logging
import emoji
import schedule
import time

# JSON data read
with open ('./stock_data.json',encoding = 'UTF-8') as json_file:
    json_data = json.load(json_file)
    json_data_2 = json_data['stock']['dataValues']

# Active Bot with the token
token = "1461066934:AAHQLhV3b7PJ-EJX-JNJdYv0tz2fZBnWmhs"
bot = telegram.Bot(token)

# updater 
updater = Updater(token=token, use_context=True)
dispatcher = updater.dispatcher

# Pre-requirement Data for send_stock_recommendation
url = "https://finance.naver.com/item/main.nhn?code=" 
ment1 = "<<  끝투's 오늘의 추천 종목  >>\n\n"
ment = ment1
image = 'test_image.jpg'

# help page
help_text = "<<  끝투 사용방법  >>\n\n1.  /help : 도움말 페이지 \n\n2.  /recommend : 오늘의 추천 주식\n\n3.  buy : 주식구매\n"
greeting_text = "끝투에 오신 것을 환영합니다!!!\n"

# Getting Stock Information
def get_stock_price(code):
    result = requests.get(url+code)
    bs_obj = BeautifulSoup(result.content,"html.parser")
    no_today = bs_obj.find("p", {"class": "no_today"})
    blind = no_today.find("span", {"class": "blind"}) # 태그 span, 속성값 blind 찾기
    now_price = blind.text
    return now_price

# make stock name as same length with each other
def equalizer(a):
    gong = ""
    for i in range (9-a):
        gong = gong + "  "
    return gong    

# make 'ment' for sending telegram
def send_stock_recommendation(json_data_2):
    now = time.localtime()
    url = "https://finance.naver.com/item/main.nhn?code=" 
    ment1 = "<<  끝투's 오늘의 추천 종목  >>\n"
    ment2 =  "현재시각 : %04d/%02d/%02d %02d:%02d:%02d\n\n" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
    ment = ment1 + ment2
    image = 'test_image.jpg'    
    for idx,info in enumerate(json_data_2):
        code = info['stock_number']
        code_name = info['stock_name']
        name_len_a = len(code_name)
        price = info['stock_price']
        price = str(price)
        price = price.rjust(8)
        url_with_code = url + code
        updated_url = "[" + code_name + "]" + "(" + url_with_code + ")"
        temp = str(idx) + '. ' + updated_url + equalizer(name_len_a) +'가격'+price + '    매수 클릭\n\n'   ####you can use get_stock_price(code) for the price
        ment = ment + temp
    # send message to users     
    bot.sendMessage(chat_id = 1437875774,text = ment, parse_mode = 'Markdown',disable_web_page_preview=True)

# Defalult Greeting    
bot.send_photo(chat_id = 1437875774, photo=open(image, 'rb'))
bot.sendMessage(chat_id = 1437875774, text= greeting_text)
bot.sendMessage(chat_id = 1437875774, text= help_text)    
    
def report():
    send_stock_recommendation(json_data_2)

# help page    
def help(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text= help_text)

start_handler = CommandHandler('help', help)
dispatcher.add_handler(start_handler)

def recommend(update, context):
    send_stock_recommendation(json_data_2)

start_handler = CommandHandler('recommend', recommend)
dispatcher.add_handler(start_handler)

# start
def start(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text="I'm a bot, please talk to me!")

start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

# def telegram_bot_sendtext(bot_message):
#     bot_token = '1461066934:AAHQLhV3b7PJ-EJX-JNJdYv0tz2fZBnWmhs'
#     bot_chatID = '1437875774'
#     send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
#     response = requests.get(send_text)
#     return response.json()

# def report():
#     my_balance = 10   ## Replace this number with an API call to fetch your account balance
#     my_message = "Current balance is: {}".format(my_balance)   ## Customize your message
#     telegram_bot_sendtext(my_message)


# schedule.every(2).minutes.do(report)
schedule.every(10).seconds.do(report)

while True:
    schedule.run_pending()
    updater.start_polling(timeout=3, clean=True)
    updater.idle()  
    time.sleep(1)

{'stock_number': '005930', 'stock_name': '삼성전자', 'stock_price': '57500', 'stock_percent': '0.67'}
{'stock_number': '000660', 'stock_name': '하이닉스', 'stock_price': '85600', 'stock_percent': '0.63'}
{'stock_number': '005380', 'stock_name': '현대차', 'stock_price': '177500', 'stock_percent': '0.59'}
{'stock_number': '051910', 'stock_name': 'LG화학', 'stock_price': '705000', 'stock_percent': '0.57'}
{'stock_number': '035720', 'stock_name': '카카오', 'stock_price': '3655', 'stock_percent': '0.54'}


NameError: name 'df' is not defined